In [12]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix, kron, eye, diags
from scipy.sparse.linalg import expm_multiply
import time
from numpy import linalg
import scipy.sparse as sparse
from scipy.linalg import expm, sinm, cosm

import warnings
warnings.filterwarnings("ignore")

def paulixyz():
    I= sparse.csr_matrix([[1., 0.],[0., 1.]]) 
    X= sparse.csr_matrix([[0., 1.],[1., 0.]]) 
    Y= sparse.csr_matrix([[0.,-1j],[1j,0.]]) 
    Z= sparse.csr_matrix([[1., 0],[0, -1.]])
    return I,X,Y,Z

I,X,Y,Z = paulixyz()

def mkron(A,B,*argv):
    K=sparse.kron(A,B,'csr')
    for arg in argv: 
        K=sparse.kron(K,arg,'csr')
    return K


def random_state(L):
    #Random normalized state
    Psi=np.random.rand(2**L,1)
    Psi=Psi/linalg.norm(Psi)
    return Psi

######### TRANSLATION OPERATOR #########

def find_j(necklace,numElts):

    for i in range(numElts):
        if necklace[numElts-1-i] >0.1:
            return numElts-1-i

    return -1

def theta_funct(necklace,numElts):

    j = find_j(necklace,numElts)

    necklace[j] = 0

    for t in range(1,numElts-j):
        necklace[j+t] = necklace[t-1]

    return int(j)

def findNecklaces(numElts):

    necklaces = []

    necklace = [1]*numElts
    necklaces.append(necklace.copy())

    for i in range(2**numElts):

        j = theta_funct(necklace,numElts)

        if numElts%(j+1)==0:
            necklaces.append(necklace.copy())

        if np.round(np.sum(necklace))==0:
            return necklaces 

# should be lists of 0 or 1's
# returns true if they are the same, else false
def compareStrings(str1,str2):

    length = len(str1)

    for i in range(length):

        if np.abs(str1[i]-str2[i]) > 0.1:
            return False

    return True

# Tells you how many times you can translate the necklace before if it repeats itself
def necklaceCycle(necklace,numElts):

    for i in range(1,numElts):

        # The period must divide the number of sites
        if numElts%i != 0:
            continue 

        if compareStrings(necklace,np.roll(necklace,i)):
            return i

    return numElts


def listToKet(ketList,numElts,roll=0):

    if ketList[(-roll)%numElts]<0.1:
        ket = [1,0]#fock(2,1)
    else:
        ket = [0,1]#fock(2,0)

    for i in range(1,numElts):
        if ketList[(i-roll)%numElts]<0.1:
            ket = sparse.kron(ket,[1,0],'csc') #tensor(ket,fock(2,1)) 
        else:
            ket = sparse.kron(ket,[0,1],'csc') #tensor(ket,fock(2,0))

    # ket = ket/ket.norm()

    return ket

def necklaceToBlochState(necklace,k,numElts,cycle=-1):

    # Check if its all zeros or all ones, need to handle those cases separately.
    # if np.sum(np.abs(np.roll(necklace,1)-necklace))<0.1:
    #     return listToKet(necklace,numElts)

    if cycle==-1:
        cycle = necklaceCycle(necklace,numElts)

    if (cycle*k)%numElts!=0:
        print('Error: Invalid k=',k,' for necklace ',necklace)
        return -1

    ket = listToKet(necklace,numElts)/np.sqrt(cycle)

    for i in range(1,cycle):
        ket += np.exp(2*np.pi*1j*k*i/numElts)*listToKet(necklace,numElts,roll=i)/np.sqrt(cycle)

    # if ket.norm()<0.1:
    #     print('ERROR: zero norm for ',necklace,'with k=',k)
    # else:
    #     norm = ket.norm()
    #     print(norm)
        # ket = ket/norm
        # print(k,necklace,ket.norm())

    return ket

#T translates to the left????
def translation_op(L):
    
    dim=2**L;
    T=np.zeros((dim,dim));
    binaryList=np.zeros((L,L));
    
    for i in range(dim):
        binaryStr=de2bi(i,L);
        T=T+np.dot(np.conj(listToKet(np.roll(binaryStr,1),L)).T,listToKet(binaryStr,L))
    
    return T

#### tensordot library 

def inv(perm):
    inverse = [0] * len(perm)
    for i, p in enumerate(perm):
        inverse[p] = i
    return inverse

def tensor_legs_onesite(i,L):
    legs=np.linspace(0,L-1,L,dtype=int).tolist() 
    legs.remove(i)
    legs.insert(0,i)
    return legs

def tensor_op_singlesite(op,vec,i,L):
    dim=2**L
    legs=tensor_legs_onesite(i,L)
    vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
    vec=vec.transpose(legs)
    vec=np.reshape(vec,[2**1,2**(L-1)])
    vec=np.tensordot(op,vec,axes=([1,0]))
    vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
    vec=vec.transpose(inv(legs))
    vec=np.reshape(vec,[dim])
    return vec.transpose()

def tensor_legs_foursite(i,j,k,l,L):
    legs=np.linspace(0,L-1,L,dtype=int).tolist() 
    legs.remove(i)
    legs.remove(j)
    legs.remove(k)
    legs.remove(l)
    legs.insert(0,i)
    legs.insert(1,j)
    legs.insert(2,k)
    legs.insert(3,l)
    return legs

def tensor_op4site_tot_evenlayer(op,vec,L):
    dim=2**L
    vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
    for i in range(0,L-1,4):
        legs=tensor_legs_foursite(i,i+1,i+2,i+3,L)
        vec=vec.transpose(legs)
        vec=np.reshape(vec,[2**4,2**(L-4)])
        vec=np.tensordot(op,vec,axes=([1,0]))
        vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
        vec=vec.transpose(inv(legs))
    vec=np.reshape(vec,[dim])
    return vec.transpose()

def tensor_op4site_tot_oddlayer(op,vec,L):
    dim=2**L
    vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
    for i in range(0,L-1,4):
        legs=tensor_legs_foursite(np.mod(i+2,L),np.mod(i+3,L),np.mod(i+4,L),np.mod(i+5,L),L)
        vec=vec.transpose(legs)
        vec=np.reshape(vec,[2**4,2**(L-4)])
        vec=np.tensordot(op,vec,axes=([1,0]))
        vec=np.reshape(vec,list(2*np.ones(L,dtype=int)))
        vec=vec.transpose(inv(legs))
    vec=np.reshape(vec,[dim])
    return vec.transpose()


######### fwht and SWAP

import numba

@numba.jit()
def fwht(a) -> None:
    """In-place Fast Walsh–Hadamard Transform of array a."""
    h = 1
    while h < len(a):
        for i in range(0, len(a), h * 2):
            for j in range(i, i + h):
                x = a[j]
                y = a[j + h]
                a[j] = x + y
                a[j + h] = x - y
        h *= 2
        
def bin2int(b):
    L = len(b)
    n = 0
    for i in range(L):
        if b[i] == 1:
            n += 2 **(L - i - 1)
    return n

def de2bi(n,L):
    a=bin(n)[2:].zfill(L)
    a=np.array(list(a), dtype=int)
    return a


def int_to_state(n, L):
   #'''generates the n'th parity eigenstate
    # by taking binary representation of n and
   # translating 0 -> |+> and 1 -> |->'''
    b = np.binary_repr(n,width=L)
    state = [1,0] if b[0]=='0' else [0,1]
    for j in range(1,L):
        if b[j]=='0':
            state = sparse.kron(state,[1,0],'csc')
        else:
            state = sparse.kron(state,[0,1],'csc')
    return state


# Swap function 
def swapFirstLast(newList): 
    newList[0], newList[-1] = newList[-1], newList[0] 
    return newList 

def swap_ij(List,i,j):
    List[i],List[j] = List[j],List[i]
    return List

def gen_SWAP_perm(L):
    dim=2**L
    basis=gen_basis(L)
    perm_even=np.zeros(dim,dtype=int)
    perm_odd=np.zeros(dim,dtype=int)
    for n in range(dim):
        basis_temp=basis[n,:]
        basis_swapped_even=basis_temp.copy()
        basis_swapped_odd=basis_temp.copy()
        for i in range(0,L,4):
            basis_swapped_even=swap_ij(basis_swapped_even,np.mod(i,L),np.mod(i+2,L))
            basis_swapped_odd=swap_ij(basis_swapped_odd,np.mod(i+2,L),np.mod(i+2+2,L))
        index_even=bin2int(basis_swapped_even)
        index_odd=bin2int(basis_swapped_odd)
        perm_even[n]=index_even
        perm_odd[n]=index_odd
    return perm_even, perm_odd

def gen_SWAP_perm(L):
    dim=2**L
    basis=gen_basis(L)
    perm_even=np.zeros(dim,dtype=int)
    perm_odd=np.zeros(dim,dtype=int)
    for n in range(dim):
        basis_temp=basis[n,:]
        basis_swapped_even=basis_temp.copy()
        basis_swapped_odd=basis_temp.copy()
        for i in range(0,L,4):
            basis_swapped_even=swap_ij(basis_swapped_even,np.mod(i,L),np.mod(i+2,L))
            basis_swapped_odd=swap_ij(basis_swapped_odd,np.mod(i+2,L),np.mod(i+2+2,L))
        index_even=bin2int(basis_swapped_even)
        index_odd=bin2int(basis_swapped_odd)
        perm_even[n]=index_even
        perm_odd[n]=index_odd
    return perm_even, perm_odd

####### basic setup 

def gen_basis(L):
    dim=2**L
    basis=np.zeros((dim,L),dtype=int)
    ind = np.arange(dim)
    for i in range(L):
        basis[:,i] = np.mod(ind,2)
        ind //=2
    return np.fliplr(basis)

def gen_Zbasis(L):
    return 1-2*gen_basis(L)

def gen_Zbasis_partial(L,vec):
    basis=gen_basis(L)
    return 1-2*basis[:,vec]
        
def gen_Ztot(L,h):
    #h is onsite disorder
    basis=gen_basis(L)
    Z = 1-2*basis
    return np.dot(Z,h)

def gen_Zpartial(L,h,vec):
    dim=2**L
    #h is onsite disorder
    #generate onsite Z for sites in vec
    basis=gen_basis(L)
    Zpartial=1-2*basis[:,vec]
    return np.dot(Zpartial,h)

def gen_ZZtot_obc(L,h):
    Z=gen_Zbasis(L)
    ZZ = Z[:,1:]*Z[:,:-1]
    return np.dot(ZZ,h)

def gen_ZZtot_pbc(L,h):
    dim=2**L
    Z=gen_Zbasis(L)
    ZZ_bulk = Z[:,1:]*Z[:,:-1]
    ZZ_boundary = Z[:,0]*Z[:,-1]
    ZZ = np.zeros((dim,L))
    ZZ[:,:-1]=ZZ_bulk
    ZZ[:,-1]=ZZ_boundary
    return np.dot(ZZ,h)

def gen_ZZtot_block(Zblock,h):
    ZZ_bulk = Zblock[1:]*Zblock[:-1]
    ZZ_boundary = Zblock[0]*Zblock[-1]
    ZZ = np.zeros(len(Zblock))
    ZZ[:-1]=ZZ_bulk
    ZZ[-1]=ZZ_boundary
    return np.sum(h*ZZ)

## will need to roll things for odd sites
def gen_ZZtot_pbc_shift(L,h,shift):
    dim=2**L
    Z=gen_Zbasis(L)
    ZZ_bulk = Z[:,1:]*Z[:,:-1]
    ZZ_boundary = Z[:,0]*Z[:,-1]
    ZZ = np.zeros((dim,L))
    ZZ[:,:-1]=ZZ_bulk
    ZZ[:,-1]=ZZ_boundary
    print(np.roll(ZZ,shift,1))
    return np.dot(np.roll(ZZ,shift,1),h)

def gen_ZZ3blocks(Zstring_array,L,blocksize,h):
    dim=2**L
    Zval=np.zeros(dim)
    for n in range(dim):
        Z3block_array=[Zstring_array[n,i:i+blocksize] for i in range(0, len(Zstring_array[n,:]), blocksize)]
        for i in range(len(Z3block_array)):
            Zval[n]+=gen_ZZtot_block(Z3block_array[i],h)
    return Zval

def gen_CP3blocks(Zstring_array,L,blocksize,phi):
    dim=2**L
    Zval=np.zeros(dim)
    for n in range(dim):
        Z3block_array=[Zstring_array[n,i:i+blocksize] for i in range(0, len(Zstring_array[n,:]), blocksize)]
        for i in range(len(Z3block_array)):
            Z=Z3block_array[i]
            Zval[n]+=phi[0]*(Z[0]*Z[1]-(Z[0]+Z[1])+1)+phi[1]*(Z[1]*Z[2]-(Z[1]+Z[2])+1)+phi[2]*(Z[2]*Z[0]-(Z[2]+Z[0])+1)
    return Zval


###### specific to triunitary circuit

def UCP_MBL_noSWAP(phi,epsilon,hz,hzz):
    I,X,Y,Z=paulixyz();
    T=translation_op(3);
    CP12=np.diag([1,1,1,1,1,1,np.exp(1j*phi[0]),np.exp(1j*phi[0])])
    CP23=np.diag([1,1,1,np.exp(1j*phi[1]),1,1,1,np.exp(1j*phi[1])])
    CP31=np.diag([1,1,1,1,1,np.exp(1j*phi[2]),1,np.exp(1j*phi[2])])
    U=CP12@CP23@CP31@expm(-1j*(hz[0]*mkron(Z,I,I)+hz[1]*mkron(I,Z,I)+hz[2]*mkron(I,I,Z)))@expm(-1j*(hzz[0]*mkron(Z,Z,I)+hzz[1]*mkron(I,Z,Z)+hzz[2]*mkron(Z,I,Z)))@expm(-1j*epsilon*(mkron(X,I,I)+mkron(I,X,I)+mkron(I,I,X)))
    return U

def UCP_MBL(phi,epsilon,hz,hzz):
    I,X,Y,Z=paulixyz();
    T=translation_op(3);
    VJ_13=T@mkron(I,np.exp(1j*np.pi/4)*expm(-1j*(np.pi/4)*(mkron(X,X)+mkron(Y,Y)+mkron(Z,Z))))@np.conj(T).T
    CP12=np.diag([1,1,1,1,1,1,np.exp(1j*phi[0]),np.exp(1j*phi[0])])
    CP23=np.diag([1,1,1,np.exp(1j*phi[1]),1,1,1,np.exp(1j*phi[1])])
    CP31=np.diag([1,1,1,1,1,np.exp(1j*phi[2]),1,np.exp(1j*phi[2])])
    U=VJ_13@CP12@CP23@CP31@expm(-1j*(hz[0]*mkron(Z,I,I)+hz[1]*mkron(I,Z,I)+hz[2]*mkron(I,I,Z)))@expm(-1j*(hzz[0]*mkron(Z,Z,I)+hzz[1]*mkron(I,Z,Z)+hzz[2]*mkron(Z,I,Z)))@expm(-1j*epsilon*(mkron(X,I,I)+mkron(I,X,I)+mkron(I,I,X)))
    return U

def gen_Udiag(L,hz_block,hzz_block,phi_block):
    dim=2**L
    blocksize=3
    hz_full=np.tile(hz_block,int(np.floor(L//(blocksize+1))))
    ##
    keepsites_even=np.arange(0,L,1).tolist()
    keepsites_odd=np.arange(0,L,1).tolist() 
    #remove sites on which there's an identity
    for i in range(3,L,4):
        keepsites_even.remove(i)
        keepsites_odd.remove(np.mod(i+2,L))
    ## even layer
    Zstring_array=gen_Zbasis_partial(L,keepsites_even)
    Uz_even=np.exp(-1j*gen_Zpartial(L,hz_full,keepsites_even))
    Uzz_even=np.exp(-1j*gen_ZZ3blocks(Zstring_array,L,blocksize,hzz_block))
    UCP_even=np.exp(1j*gen_CP3blocks(Zstring_array,L,blocksize,phi_block)/4)

    ## odd layer
    Zstring_array=np.roll(gen_Zbasis_partial(L,keepsites_odd),2,axis=1) #need to roll it by 2!
    Uz_odd=np.exp(-1j*gen_Zpartial(L,np.roll(hz_full,1),keepsites_odd)) #need to roll it by 1!
    Uzz_odd=np.exp(-1j*gen_ZZ3blocks(Zstring_array,L,blocksize,hzz_block))
    UCP_odd=np.exp(1j*gen_CP3blocks(Zstring_array,L,blocksize,phi_block)/4)

    ##
    U_diag_even=UCP_even*Uz_even*Uzz_even
    U_diag_odd=UCP_odd*Uz_odd*Uzz_odd
    U_diag_tot=UCP_odd*UCP_even*Uz_odd*Uz_even*Uzz_odd*Uzz_even

    return U_diag_tot,U_diag_even,U_diag_odd

def gen_Ux(L,eps):
    blocksize=3
    hx_block=eps*np.ones(blocksize)
    hx_full=np.tile(hx_block,int(np.floor(L//(blocksize+1))))
    ##
    keepsites_even=np.arange(0,L,1).tolist()
    keepsites_odd=np.arange(0,L,1).tolist() 
    #remove sites on which there's an identity
    for i in range(3,L,4):
        keepsites_even.remove(i)
        keepsites_odd.remove(np.mod(i+2,L))
    ##
    Ux_zbasis_even=np.exp(-1j*gen_Zpartial(L,hx_full,keepsites_even))
    Ux_zbasis_odd=np.exp(-1j*gen_Zpartial(L,np.roll(hx_full,1),keepsites_odd)) #need to roll it by 1!
    Ux_zbasis_tot=Ux_zbasis_odd*Ux_zbasis_even
    return Ux_zbasis_tot, Ux_zbasis_even, Ux_zbasis_odd

def gen_triunitary_evolution(L,psi,Udiag_even,Udiag_odd,Ux_zbasis_even,Ux_zbasis_odd,):
    dim=2**L
    perm_even,perm_odd=gen_SWAP_perm(L)
    fwht(psi)
    psi=Ux_zbasis_even*psi
    fwht(psi)
    psi=Udiag_even*psi
    fwht(psi)
    psi=Ux_zbasis_odd*psi
    fwht(psi)
    psi=Udiag_odd*psi
    psi=psi/1./dim**2
    return psi

def gen_triunitary_evolution_wSWAP(L,psi,Udiag_even,Udiag_odd,Ux_zbasis_even,Ux_zbasis_odd,perm_even,perm_odd):
    dim=2**L
    fwht(psi)
    psi=Ux_zbasis_even*psi
    fwht(psi)
    psi=Udiag_even*psi
    psi=psi[perm_even]
    fwht(psi)
    psi=Ux_zbasis_odd*psi
    fwht(psi)
    psi=Udiag_odd*psi
    psi=psi/1./dim**2
    psi=psi[perm_odd]
    return psi

# Comparing tensordot and fwht evolution times

In [14]:
## Let's check for multiple time steps!

#####################################################################################

L=20
dim=2**L
R=10

#perm_even=np.load('perm_even_L%d.npy' %L)
#perm_odd=np.load('perm_odd_L%d.npy' %L)
perm_even,perm_odd=gen_SWAP_perm(L)

I,sx,sy,sz = paulixyz()

eps=0.1
W=2*np.pi
psi=random_state(L).flatten()
phi=np.random.uniform(0,2*np.pi,3)
hz=np.random.uniform(0,W,3)
hzz=np.random.uniform(0,W,3)

tvec=np.arange(1,500,1)
C12_tensordot=np.zeros(len(tvec))
C12_fwht=np.zeros(len(tvec))
time_tensordot=[]
time_fwht=[]

## tensordot
u123=UCP_MBL(phi,eps,hz,hzz)
u1234=mkron(u123,I)

psi1_tensordot=psi.copy()
psi2_tensordot=psi.copy()
psi2_tensordot=tensor_op_singlesite(sz.todense(),psi2_tensordot,3,L)

psi1_fwht=psi.copy()
psi2_fwht=psi.copy()
psi2_fwht=tensor_op_singlesite(sz.todense(),psi2_fwht,3,L)


for t in tvec:
    a2=time.time()
    psi1_tensordot=tensor_op4site_tot_evenlayer(u1234.todense(),psi1_tensordot,L)
    psi1_tensordot=tensor_op4site_tot_oddlayer(u1234.todense(),psi1_tensordot,L)
    psi2_tensordot=tensor_op4site_tot_evenlayer(u1234.todense(),psi2_tensordot,L)
    psi2_tensordot=tensor_op4site_tot_oddlayer(u1234.todense(),psi2_tensordot,L)
    C12_tensordot[t-1]=np.dot(np.conj(tensor_op_singlesite(sz.todense(),psi1_tensordot,3,L)).T,psi2_tensordot)
    b2=time.time()
    time_tensordot.append(b2-a2)

## fwht
Ux_zbasis_tot,Ux_zbasis_even,Ux_zbasis_odd=gen_Ux(L,eps)
Udiag_tot,Udiag_even,Udiag_odd=gen_Udiag(L,hz,hzz,phi)


for t in tvec:
    a3=time.time()
    psi1_fwht=gen_triunitary_evolution_wSWAP(L,psi1_fwht,Udiag_even,Udiag_odd,Ux_zbasis_even,Ux_zbasis_odd,perm_even,perm_odd)
    psi2_fwht=gen_triunitary_evolution_wSWAP(L,psi2_fwht,Udiag_even,Udiag_odd,Ux_zbasis_even,Ux_zbasis_odd,perm_even,perm_odd)
    C12_fwht[t-1]=np.dot(np.conj(tensor_op_singlesite(sz.todense(),psi1_fwht,3,L)).T,psi2_fwht)
    b3=time.time()
    time_fwht.append(b3-a3)


diff_tensordot_fwht=np.round(C12_fwht-C12_tensordot,4)
print(np.nonzero(diff_tensordot_fwht))
print(['time tensordot= ', np.mean(time_tensordot)])
print(['time fwht= ', np.mean(time_fwht)])


(array([], dtype=int64),)
['time tensordot= ', 0.2823723361105145]
['time fwht= ', 0.2917016565441369]


In [10]:
%load_ext line_profiler

In [11]:
%lprun -f gen_triunitary_evolution_wSWAP gen_triunitary_evolution_wSWAP(L,psi1_fwht,Udiag_even,Udiag_odd,Ux_zbasis_even,Ux_zbasis_odd,perm_even,perm_odd)

In [4]:
L=20

a=time.time()
perm_even_L20,perm_odd_L20=gen_SWAP_perm(L)
b=time.time()
print(b-a)

45.72426509857178
